In [26]:
print(1)

1


In [27]:
pip install -U "autogen-ext[ollama]" 

You should consider upgrading via the 'c:\Akshay\Langchain\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [28]:
from autogen_core.models import UserMessage
from autogen_ext.models.ollama import OllamaChatCompletionClient

# Assuming your Ollama server is running locally on port 11434.
ollama_model_client = OllamaChatCompletionClient(model="llama3.2")

response = await ollama_model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(response)
await ollama_model_client.close()


finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=32, completion_tokens=8) cached=False logprobs=None thought=None


In [29]:
pip install autogen-agentchat

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Akshay\Langchain\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


## create confluence agent

In [34]:
pip install pyautogen atlassian-python-api

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Akshay\Langchain\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [90]:
from autogen_core.models import UserMessage

# Get question from user input
question = input("Enter your question for Llama: ")

if not question.strip():
    question = "What are the top 3 features of Llama 3.2?"
    print(f"No input provided. Using default: {question}\n")
else:
    print()

print(f"Question: {question}\n")
print("Sending to Llama 3.2...")

# Query Llama and get response
response = await ollama_model_client.create([UserMessage(content=question, source="user")])
answer = response.content

print(f"\nLlama Response:\n{answer}")


Question: what is AI, explain in short?

Sending to Llama 3.2...

Llama Response:
AI stands for Artificial Intelligence. It refers to the development of computer systems that can perform tasks that typically require human intelligence, such as:

* Learning
* Problem-solving
* Reasoning
* Perception
* Language understanding

In short, AI aims to create machines that can think and act like humans, making them more efficient, accurate, and helpful in various aspects of life.


In [91]:
# Publish the response to Confluence
print("\nPublishing to Confluence...\n")
result = publish_to_confluence(answer)
print(result)


Publishing to Confluence...

✓ Successfully created new page 'Agent Report' at https://aamreakshay.atlassian.net/wiki/spaces/SD/pages/1081358


In [92]:
from atlassian import Confluence
import os

# Base URL for Confluence API (must include /wiki for Cloud instances)
url="https://aamreakshay.atlassian.net/wiki"
username="aamre.akshay@gmail.com"
api_token="ATATT3xFfGF0foGnplYlwaAFXThUIPN_maK9JFGDV2BkZ1bwk7jE73M_y6vD5L7rpI9jN4Y37tFrPNb8X9Uh0gus6r6SaMTECqtlSGKnr27WL3ovF9Ab-CxzeEim_I53xzNLMIFGHIE-TNM-knQHhdEcPXPEzjzd1biYD5fbrM5bTXRk-02pEEs=05479591"
space_key="SD"
page_id=622593  # Your SD space page ID

confluence = Confluence(url=url, username=username, password=api_token)

In [ ]:
from atlassian import Confluence

# Initialize the Confluence client
# Ensure your URL, email, and API Token are correct
print(f"Connecting to: {url}")
print(f"Username: {username}")
print(f"Token length: {len(api_token)} characters")

try:
    confluence = Confluence(
        url=url,
        username=username,
        password=api_token, 
        cloud=True
    )
    print("✓ Confluence client created")
except Exception as e:
    print(f"✗ Failed to create client: {e}")

def publish_to_confluence(content: str) -> str:
    """
    Publishes preformatted Confluence storage-format HTML to the "Agent Report" page.

    Args:
        content: The content in Confluence storage-format (HTML) string.

    Returns:
        A success or error message with the page URL.
    """
    try:
        page_title = "Agent Report"
        existing_pages = confluence.get_all_pages_from_space(space_key)
        existing_page = next((p for p in existing_pages if p['title'] == page_title), None)

        if existing_page:
            page_id = existing_page['id']
            confluence.update_page(
                page_id=page_id,
                title=page_title,
                body=content,
                minor_edit=False
            )
            page_url = f"{url}/spaces/{space_key}/pages/{page_id}"
            return f"✓ Successfully updated page '{page_title}' at {page_url}"
        else:
            response = confluence.create_page(
                space=space_key,
                title=page_title,
                body=content,
                representation='storage'
            )
            page_id = response['id']
            page_url = f"{url}/spaces/{space_key}/pages/{page_id}"
            return f"✓ Successfully created new page '{page_title}' at {page_url}"
    except Exception as e:
        return f"✗ Error publishing to Confluence: {str(e)}"


Connecting to: https://aamreakshay.atlassian.net/wiki
Username: aamre.akshay@gmail.com
Token length: 192 characters
✓ Confluence client created


In [94]:
# Test connection and list pages in SD space
try:
    print("✓ Testing Confluence connection...")
    pages = confluence.get_all_pages_from_space(space_key)
    print(f"✓ Successfully connected to space '{space_key}'")
    print(f"✓ Found {len(pages)} pages in this space")
    
    # Check if "Agent Report" page exists
    agent_report = next((p for p in pages if p['title'] == 'Agent Report'), None)
    if agent_report:
        print(f"\n✓ 'Agent Report' page exists (ID: {agent_report['id']})")
        print(f"  Will update this page when agent runs")
    else:
        print(f"\n✓ 'Agent Report' page does not exist yet")
        print(f"  Will create it when agent runs")
        
except Exception as e:
    print(f"✗ Error: {str(e)}")
    print("\nMake sure:")
    print(f"1. API token is valid")
    print(f"2. Space key '{space_key}' exists and you have access")
    print(f"3. URL is correct: {url}")


✓ Testing Confluence connection...
✓ Successfully connected to space 'SD'
✓ Found 6 pages in this space

✓ 'Agent Report' page exists (ID: 1081358)
  Will update this page when agent runs


In [95]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console

# Setup agent with Confluence tool (optional - for more complex queries)
agent = AssistantAgent(
    name='confluence_agent',
    model_client=ollama_model_client,
    tools=[publish_to_confluence],
    system_message='You are a helpful assistant. When asked to publish content, use the publish_to_confluence tool.',
)

# For advanced use: Let agent handle the task and publishing automatically
# Uncomment below to use agent mode instead of direct publishing
# await Console(agent.run_stream(task="Research Llama 3.2 features and publish a summary to Confluence"))


In [ ]:
# Read stored procedure from sp.txt, generate documentation, and publish to Confluence
import re
import html as _html
from pathlib import Path

sp_path = Path(r"c:\Akshay\Langchain\AgenticAI\sp.txt")
if not sp_path.exists():
    print(f"sp.txt not found at {sp_path}")
else:
    sp = sp_path.read_text(encoding='utf-8')

    # Extract procedure name
    m = re.search(r"CREATE\s+PROCEDURE\s+(?:dbo\.)?([^(\s]+)", sp, re.IGNORECASE)
    proc_name = m.group(1) if m else "StoredProcedure"

    # Extract parameters block (between procedure line and AS)
    pm = re.search(r"CREATE\s+PROCEDURE[\s\S]*?\n(.*?)\nAS", sp, re.IGNORECASE)
    params_block = pm.group(1) if pm else ""
    params = []
    for line in params_block.splitlines():
        line = line.strip().rstrip(',')
        if line.startswith('@'):
            parts = line.split()
            pname = parts[0]
            ptype = parts[1] if len(parts) > 1 else ''
            params.append((pname, ptype))

    # Extract inline comments as steps
    steps = re.findall(r"--\s*(.*)", sp)

    # Detect referenced tables
    tables = set(re.findall(r"\b(?:FROM|INTO|UPDATE|INSERT\s+INTO)\s+([A-Za-z0-9_]+)", sp, re.IGNORECASE))

    # Build documentation as Confluence storage-format HTML
    html_parts = []
    html_parts.append(f"<h1>{_html.escape(proc_name)} Documentation</h1>")
    html_parts.append(f"<h2>Purpose</h2>")
    purpose = ("Processes a department store sale: validates product availability, deducts inventory, records the sale, "
               "updates customer loyalty points, logs the transaction for auditing, and handles errors.")
    html_parts.append(f"<p>{_html.escape(purpose)}</p>")

    # Parameters table
    html_parts.append("<h2>Parameters</h2>")
    if params:
        tbl = ["<table>", "<tr><th>Parameter</th><th>Type</th></tr>"]
        for pname, ptype in params:
            tbl.append(f"<tr><td>{_html.escape(pname)}</td><td>{_html.escape(ptype)}</td></tr>")
        tbl.append("</table>")
        html_parts.extend(tbl)
    else:
        html_parts.append("<p>No parameters detected.</p>")

    # Process flow
    html_parts.append("<h2>Process Flow</h2>")
    if steps:
        html_parts.append("<ol>")
        for s in steps:
            html_parts.append(f"<li>{_html.escape(s)}</li>")
        html_parts.append("</ol>")
    else:
        html_parts.append("<p>Could not extract inline steps from the procedure.</p>")

    # Tables affected
    html_parts.append("<h2>Tables Affected</h2>")
    if tables:
        html_parts.append("<ul>")
        for t in sorted(tables):
            html_parts.append(f"<li>{_html.escape(t)}</li>")
        html_parts.append("</ul>")
    else:
        html_parts.append("<p>No table references detected.</p>")

    # Sample call
    html_parts.append("<h2>Sample Call</h2>")
    sample = ("EXEC dbo." + proc_name + " @CustomerID = 123, @ProductID = 456, @Quantity = 2, @StoreID = 1, @SalesPersonID = 10, @TransactionStatus = @status OUTPUT;")
    html_parts.append(f"<pre><code>{_html.escape(sample)}</code></pre>")

    # Full procedure
    html_parts.append("<h2>Stored Procedure Source</h2>")
    html_parts.append(f"<pre><code>{_html.escape(sp)}</code></pre>")

    final_html = "\n".join(html_parts)

    # Publish
    print("Publishing documentation to Confluence...")
    res = publish_to_confluence(final_html)
    print(res)
